In [1]:
import numpy as np
from scipy import optimize
from __future__ import division

In [3]:
import numpy as np
import sklearn
import sklearn.datasets
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

class Network(object):
    
    def __init__(self, sizes):
        sizes_offset = zip(sizes[:-1], sizes[1:])

        self.weights = [np.random.randn(s1, s2) for s1, s2 in sizes_offset]
        self.biases = [np.random.randn(1, s) for s in sizes[1:]]

    def predict(self, X):
        activations, _ = self._feed_forward(X)
        return np.argmax(activations[-1], axis=1)

    def cost(self, X, y):

        activations, _ = self._feed_forward(X)
        pred = activations[-1]
        err = (-y) * np.log(pred) - (1 - y) * np.log(1 - pred)
        cost = np.sum(err) / len(X)

        return cost

    def _feed_forward(self, X):

        activations = [X]
        combinations = [X]

        for weights, bias in zip(self.weights, self.biases):
            combinations.append(np.dot(activations[-1], weights) + bias)
            activations.append(sigmoid(combinations[-1]))

        return activations, combinations

    def _backpropagate(self, X, y, reg):

        activations, combinations = self._feed_forward(X)
        delta_w = []
        delta_b = []
        delta = activations[-1] - y

        for j in reversed(range(len(self.weights))):
            a = activations[j]
            z = combinations[j]
            w = self.weights[j]

            dw = np.dot(a.T, delta) + reg * w
            db = np.sum(delta, axis=0)
            delta = delta.dot(w.T) * sigmoid_prime(z)

            delta_b.insert(0, db)
            delta_w.insert(0, dw)

        return delta_w, delta_b


    def fit(self, X, y, iterations=20000, alpha=0.01, reg=0.01):

        for i in range(iterations):

            delta_w, delta_b = self._backpropagate(X, y, reg)

            self.weights = [w - alpha * d for w, d in zip(self.weights, delta_w)]
            self.biases = [b - alpha * d for b, d in zip(self.biases, delta_b)]

            if i % 1000 == 0:
                print("Iteration %i: %f" %(i, self.cost(X, y)))

X, y = sklearn.datasets.make_moons(200, noise=0.10)
yy = label_binarize([i + 1 for i in y], [1, 2, 3])[:,0:2]

network = Network([X.shape[1], 10, 10, yy.shape[1]])
network.fit(X, yy)

print(accuracy_score(y, network.predict(X)))

Iteration 0: 2.111802
Iteration 1000: 0.007416
Iteration 2000: 0.003806
Iteration 3000: 0.003117
Iteration 4000: 0.002898
Iteration 5000: 0.002811
Iteration 6000: 0.002766
Iteration 7000: 0.002736
Iteration 8000: 0.002714
Iteration 9000: 0.002696
Iteration 10000: 0.002683
Iteration 11000: 0.002671
Iteration 12000: 0.002662
Iteration 13000: 0.002653
Iteration 14000: 0.002646
Iteration 15000: 0.002638
Iteration 16000: 0.002632
Iteration 17000: 0.002626
Iteration 18000: 0.002621
Iteration 19000: 0.002616
1.0


In [4]:
from sklearn.preprocessing import label_binarize

n = Network([2,4,2])
activations, sums = n.feed_forward(X)
print(activations, sums)
cost = n.cost(X, y)
print(cost)
#print(cost)
#n.backpropagate(X, y)


AttributeError: 'Network' object has no attribute 'feed_forward'

In [ ]:
import random
import numpy as np

class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(1, y) for y in sizes[1:]]
        self.weights = [np.random.randn(x, y)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
        print([b.shape for b in self.biases])
        print([w.shape for w in self.weights])
        #print(self.biases[0].shape, type(self.biases[0]))

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            print(a.shape, w.shape, b.shape)
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))